# Neural Networks (Safety category)

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import time

from sklearn.model_selection import train_test_split

from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [18]:
safety_dataset_new = pd.read_csv('safety_dataset_new.csv')
safety_dataset_new.head()

,bookingID,Accuracy_avg,Bearing_avg,acceleration_x_avg,acceleration_y_avg,acceleration_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,Speed_avg,...,Bearing_std,acceleration_x_std,acceleration_y_std,acceleration_z_std,gyro_x_std,gyro_y_std,gyro_z_std,Speed_std,time,label
0,0,10.165339,176.526099,-0.711264,-9.613822,-1.619658,0.003328,-0.006118,-0.004188,8.994822,...,129.231351,0.928022,0.639934,1.141266,0.065954,0.100225,0.063685,7.199919,1589.0,0
1,1,3.718763,124.198590,-0.525406,9.532086,-2.198999,-0.002467,-0.007540,0.000405,7.881588,...,89.861236,0.744157,0.533915,0.854271,0.027740,0.091699,0.033838,7.059362,1034.0,1
2,2,3.930626,173.794872,0.306786,9.843183,0.139347,0.006458,-0.012861,0.002597,3.157213,...,119.316520,0.756589,0.505693,1.020021,0.053903,0.117321,0.036215,2.897762,825.0,1
3,4,10.000000,151.807013,-0.365117,-9.406439,-2.613639,-0.022884,0.023232,-0.000376,6.150996,...,71.273774,0.527220,0.598023,0.779529,0.042342,0.112567,0.065927,5.595901,1094.0,1
4,6,4.586721,197.812785,0.490616,9.538043,2.355059,0.003877,0.000436,0.002930,4.628921,...,111.868249,0.826271,0.617210,0.942163,0.055170,0.106815,0.057438,5.314844,1094.0,0


In [19]:
data = safety_dataset_new.values
print(type(data), data.shape)

<class 'numpy.ndarray'> (20000, 57)


In [20]:
X = safety_dataset_new.drop(['label', 'bookingID'], axis=1)
y = safety_dataset_new.label

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [25]:
def neural_net():
    """
    input: N.A.
    output: a neural network model 
    
    purpose: define the neural network model
    """
    # initialize model
    model = models.Sequential()
    
    # Input layer
    model.add(layers.Dense(50, activation = "relu", input_shape=(55, )))
    
    # Hidden layers
    model.add(layers.Dense(100, activation = "relu"))
    model.add(layers.Dense(50, activation = "relu"))
    
    # Output Layer
    model.add(layers.Dense(1, kernel_initializer='normal'))
    
    # Compile neural network
    model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])

    return model

In [26]:
model = neural_net()

# Wrap Keras model so it can be used by scikit-learn for cross-fold validation
neural_network = KerasClassifier(build_fn=neural_net, 
                                 epochs=10, 
                                 batch_size=100, 
                                 verbose=0)

# print model information
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 50)                2800      
_________________________________________________________________
dense_14 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_15 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 51        
Total params: 13,001
Trainable params: 13,001
Non-trainable params: 0
_________________________________________________________________


In [27]:
start = time.time()

model_info = model.fit(X_train, y_train, batch_size=64, \
                       nb_epoch=20, verbose=1, validation_split=0.2)
end = time.time()
print("Model took %0.2f seconds to train"%(end - start))

C:\Users\Pinar\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 12000 samples, validate on 3000 samples
Epoch 1/20
12000/12000 [==============================] - 1s 103us/step - loss: 388.8261 - acc: 0.4644 - val_loss: 21654.9082 - val_acc: 0.0167
Epoch 2/20
12000/12000 [==============================] - 0s 34us/step - loss: 1580.7872 - acc: 0.3414 - val_loss: 179.6499 - val_acc: 0.6307
Epoch 3/20
12000/12000 [==============================] - 0s 33us/step - loss: 4451.1426 - acc: 0.5586 - val_loss: 16879.5915 - val_acc: 0.0867
Epoch 4/20
12000/12000 [==============================] - 0s 37us/step - loss: 1622.4696 - acc: 0.4952 - val_loss: 49491.1923 - val_acc: 0.0430
Epoch 5/20
12000/12000 [==============================] - 0s 35us/step - loss: 3743.6963 - acc: 0.3282 - val_loss: 1247.0865 - val_acc: 0.6780
Epoch 6/20
12000/12000 [==============================] - ETA: 0s - loss: 3448.4053 - acc: 0.54 - 0s 34us/step - loss: 3420.8216 - acc: 0.5507 - val_loss: 6875.9482 - val_acc: 0.7073
Epoch 7/20
12000/12000 [===========================

In [28]:
# Evaluate neural network using five-fold cross-validation
# Referenced from https://chrisalbon.com/deep_learning/keras/k-fold_cross-validating_neural_networks/
cross_val_score(neural_network, X_train, y_train, cv=5)

array([ 0.76066666,  0.726     ,  0.70766667,  0.747     ,  0.72333333])

In [30]:
# Evaluate model on test data
score = model.evaluate(X_test, y_test, verbose=1)

5000/5000 [==============================] - 0s 33us/step
